# PDF support

Claude can read and analyze PDF files directly, making it a powerful tool for document processing. This capability works similarly to image processing, but with a few key differences in how you structure your code.

In [1]:
# Load env variables and create client
import base64
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [2]:
# Helper functions
from anthropic.types import Message


def add_messages(messages, message, role="user"):
    message = {"role": role, "content": message.content if isinstance(message, Message) else message}
    messages.append(message)


def chat(
    messages,
    system=None,
    temperature=1.0,
    stop_sequences=[],
    tools=None,
    thinking=False,
    thinking_budget=1024,
):
    params = {
        "model": model,
        "max_tokens": 4000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if thinking:
        params["thinking"] = {
            "type": "enabled",
            "budget_tokens": thinking_budget,
        }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [4]:
with open("earth.pdf", "rb") as f:
    file_bytes = base64.standard_b64encode(f.read()).decode("utf-8")

messages = []

add_messages(messages, [
    # Image Block
    {
        "type": "document",
        "source": {
            "type": "base64",
            "media_type": "application/pdf",
            "data": file_bytes,
        }
    },
    # Text Block
    {
        "type": "text",
        "text": "Summarize the document in one sentence"
    }
],
role="user")

chat(messages)


Message(id='msg_011Ej54x63kVdG5uGwXKRu6z', content=[TextBlock(citations=None, text='This Wikipedia article provides a comprehensive overview of Earth, describing it as the third planet from the Sun and the only known celestial body to harbor life, covering its physical characteristics, formation about 4.5 billion years ago, orbital properties, atmosphere, and the development of life and human civilization on its surface.', type='text')], model='claude-sonnet-4-5-20250929', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation=CacheCreation(ephemeral_1h_input_tokens=0, ephemeral_5m_input_tokens=0), cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=9625, output_tokens=66, server_tool_use=None, service_tier='standard', inference_geo='not_available'))